In [42]:
%load_ext autoreload
%autoreload 2
import sys
from pathlib import Path
path = str(Path.cwd().parent)
print(path)
sys.path.insert(1, path)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
/home/ubuntu/varios/skforecast


In [43]:
import pandas as pd
import numpy as np
import os
from skforecast.ForecasterAutoreg import ForecasterAutoreg
from skforecast.datasets import fetch_dataset
from lightgbm import LGBMRegressor
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler

In [44]:
# Download data
# ==============================================================================
data = fetch_dataset(
    name="h2o", raw=True, kwargs_read_csv={"names": ["y", "datetime"], "header": 0}
)

# Data preprocessing
# ==============================================================================
data['datetime'] = pd.to_datetime(data['datetime'], format='%Y-%m-%d')
data = data.set_index('datetime')
data = data.asfreq('MS')
data = data['y']
data = data.sort_index()

# Split train-test
# ==============================================================================
steps = 36
data_train = data[:-steps]
data_test  = data[-steps:]

h2o
---
Monthly expenditure ($AUD) on corticosteroid drugs that the Australian health
system had between 1991 and 2008.
Hyndman R (2023). fpp3: Data for Forecasting: Principles and Practice(3rd
Edition). http://pkg.robjhyndman.com/fpp3package/,https://github.com/robjhyndman
/fpp3package, http://OTexts.com/fpp3.
Shape of the dataset: (204, 2)


In [45]:
forecaster = ForecasterAutoreg(
                 regressor = LGBMRegressor(random_state=123, verbose=-1),
                 lags          = 15,
                 differentiation=1
             )

forecaster.fit(y=data_train)

In [46]:
predictions = forecaster.predict(steps=36)
predictions.head(3)

2005-07-01    0.909529
2005-08-01    0.941633
2005-09-01    1.007650
Freq: MS, Name: pred, dtype: float64

In [47]:
# forecaster.set_out_sample_residuals(
#     residuals = data_test - predictions
# )

In [48]:
forecaster.set_out_sample_residuals(residuals=forecaster.in_sample_residuals_)

/home/ubuntu/varios/skforecast/skforecast/ForecasterAutoreg/ForecasterAutoreg.py:2054: UserWarning: Residuals are being set directly in a Forecaster with a transformation or differentiation applied. Ensure residuals are pre-transformed or pre-differentiated. Otherwise, pass `y_true` and `y_pred` instead of `residuals` to apply transformations automatically.
  warnings.warn(


In [5]:
def print_directory_tree(startpath, indent_level=0, ignore_folders=None):
    if ignore_folders is None:
        ignore_folders = ['__pycache__', 'tests']  # Default folders to ignore

    for item in os.listdir(startpath):
        path = os.path.join(startpath, item)

        # Skip ignored folders
        if os.path.isdir(path) and item in ignore_folders:
            continue

        print(' ' * indent_level + '|-- ' + item)  # Print the item with indent

        # Recursively call for subdirectories
        if os.path.isdir(path):
            print_directory_tree(path, indent_level + 4, ignore_folders)

# Example usage: Set the path of the directory you want to print
directory_path = '/home/ubuntu/varios/skforecast/skforecast'
print_directory_tree(directory_path , ignore_folders=['__pycache__', 'tests', 'htmlcov'])

|-- Sarimax
    |-- Sarimax.py
    |-- __init__.py
|-- feature_selection
    |-- feature_selection.py
    |-- __init__.py
|-- plot
    |-- plot.py
    |-- __init__.py
|-- model_selection_sarimax
    |-- test
    |-- __init__.py
    |-- model_selection_sarimax.py
|-- exceptions
    |-- exceptions.py
    |-- __init__.py
|-- ForecasterAutoregCustom
    |-- ForecasterAutoregCustom.py
    |-- __init__.py
|-- ForecasterAutoregMultiVariate
    |-- __init__.py
    |-- ForecasterAutoregMultiVariate.py
|-- metrics
    |-- metrics.py
    |-- __init__.py
|-- preprocessing
    |-- preprocessing.py
    |-- test
    |-- __init__.py
|-- ForecasterSarimax
    |-- ForecasterSarimax.py
    |-- __init__.py
|-- datasets
    |-- datasets.py
    |-- __init__.py
|-- ForecasterBaseline
    |-- ForecasterEquivalentDate.py
    |-- __init__.py
|-- model_selection
    |-- _split.py
    |-- _search.py
    |-- _validation.py
    |-- model_selection_old.py
    |-- __init__.py
|-- __init__.py
|-- ForecasterRnn
    |--

    
    |-- skforecast
        |-- autoreg
            |-- ForecasterRecursive.py                   
            |-- ForecasterMultiseriesRecursive.py
            |-- ForecasterSarimax.py
            |-- ForecasterEquivalentDate.py             
            |-- __init__.py
        |-- direct
            |-- ForecasterDirect.py
            |-- ForecasterMultivariateDirect.py
            |-- __init__.py
        |-- deep_learning
            |-- ForecasterRnn.py
        |-- base
            |-- ForecasterBase.py
        |-- __init__.py